In [1]:
import yelp_functions
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import ast
import datetime
from dateutil.rrule import rrule, MONTHLY
import matplotlib.pyplot as plt
from collections import Counter, OrderedDict
from IPython.display import clear_output, display

In [2]:
#business = pd.read_json('yelp_academic_dataset_business.json', lines=True)
AZ_reviews = pd.read_csv('AZ_review.csv')
#tips = pd.read_json('yelp_academic_dataset_tip.json', lines=True)

In [3]:
#load ye olde data
AZ_reviews = pd.read_csv('AZ_review.csv')
#assign datetime
AZ_reviews_datetimed = AZ_reviews.assign(datetime = pd.to_datetime(AZ_reviews['date'])).drop(labels=['date'], axis = 'columns').set_index('datetime').sort_index()
#sort by month
AZ_reviews_groupby_date = AZ_reviews_datetimed.groupby([AZ_reviews_datetimed.index.year.rename('year'), AZ_reviews_datetimed.index.month.rename('month')])

In [4]:
len(AZ_reviews_datetimed)

1877589

In [5]:
def add_weighted_review_avg(df,weight):
    
    dictionary = {}
    weighted_review_count = [1+(sum(x)*weight) if x != 0 else 1 for x in zip(df['funny'].values, df['cool'].values,df['useful'].values)]
    df['weighted_review_count'] = weighted_review_count
    gbym = df.groupby([df.index.year.rename('year'), df.index.month.rename('month')])
    for key,records in gbym: 
        gb_df = gbym.get_group(key)
        bids = gb_df['business_id'].unique()
        for i in bids: 
            bid_records = gb_df.loc[gb_df['business_id'] == i]
            monthly_weighted_review_cnt_per_bid = sum(bid_records['weighted_review_count'].values)/len(bid_records['weighted_review_count'].values)
            dictionary[str(key)+'%BREAK%'+str(i)] = monthly_weighted_review_cnt_per_bid
            
    
    month_date = [date_bid.split('%BREAK%')[0] for date_bid in list(dictionary.keys())]
    bids = [date_bid.split('%BREAK%')[1] for date_bid in list(dictionary.keys())]
    mnth_weight_avg_rvw_cnt = [avg for avg in dictionary.values()] 
    
    avg_weight_rvw = pd.DataFrame({"business_id":bids,
                             "monthly weighted average review count": mnth_weight_avg_rvw_cnt},index = month_date)

    avg_weight_rvw = pd.DataFrame({"business_id":bids,
                             "monthly weighted average review count": mnth_weight_avg_rvw_cnt,"month_date": month_date})
     
    groupbydate =  [str((x.year,x.month)) for x in df.index]
    df['groupbydate'] = groupbydate
    
    df = df.reset_index().merge(avg_weight_rvw, how="left",left_on = ['business_id','groupbydate'], right_on = ['business_id','month_date']).set_index('datetime')
    df.drop(['groupbydate', 'month_date','weighted_review_count'], axis=1,inplace = True)
    
    return df

In [9]:
testingdf123

,Unnamed: 0,business_id,cool,funny,review_id,stars,text,useful,user_id,monthly weighted average review count
datetime,,,,,,,,,,
2009-05-07,2622307,XMMLRvV4IMxIGyc4H37LxA,0,0,Xoh7vIglTRgtuyQlR8cwIQ,4,We tried this place for the first time this we...,0,5xCkPXiBjb6lWtUZ0Bwc0w,1.016364
2009-05-10,2623037,XMMLRvV4IMxIGyc4H37LxA,0,0,e9Ktm5PI80wH2BBsVVh0gQ,3,I'm having a hard time squelching my inner cyn...,0,4VmuXuSRhv5UxYUy3tMpiQ,1.016364
2009-05-12,2622226,XMMLRvV4IMxIGyc4H37LxA,1,0,rnwvlfQbHLFfgtM_u7B0cQ,5,Dr. Weil should come to L.A. with his restaura...,1,gd_wM1Ezz-6xtBm2je18HA,1.016364
2009-05-13,2623121,XMMLRvV4IMxIGyc4H37LxA,0,0,CpQ63ArIki0tWOgL6PBbIw,4,Have not been disappointed. Excellent food wi...,1,RS5qRwpy66w1ADVylNhX1A,1.016364
2009-05-15,2622817,XMMLRvV4IMxIGyc4H37LxA,0,0,oLI1-bwQ2D1uMqOXbPoA-A,3,If you like California fare AND California att...,0,aZFNp5IyNA11TvSrsuJ_Bg,1.016364
2009-05-16,2622234,XMMLRvV4IMxIGyc4H37LxA,0,0,i5vU0FtM4hhAZr7mT088_A,3,"We had a decent lunch here, but not good enoug...",0,5VK5zd4nQFkCj2TU32U8og,1.016364
2009-05-19,2622525,XMMLRvV4IMxIGyc4H37LxA,1,1,d16IItL4KYYuy-NpfMmTew,4,This place was just delightful. I have been h...,1,zJA8ZSsJhtQ3VE96jjhHnA,1.016364
2009-05-20,2622985,XMMLRvV4IMxIGyc4H37LxA,0,0,5zilAAG9INj30-m9nhKywQ,5,"Love this restaurant. I had the kale salad, th...",1,SjKbDhgkiGJn8oXf2AXpRw,1.016364
2009-05-22,2622118,XMMLRvV4IMxIGyc4H37LxA,1,0,BRRg519q5Esu4iLeBaR-VQ,5,True Food was a truly unique dining experience...,1,D3AKg27TpYsIiYidvVSgtg,1.016364


In [7]:
testing456 = add_weighted_review_avg(AZ_reviews_datetimed[20000:40000],.02)
testgb = testing456.groupby([testing456.index.year.rename('year'), testing456.index.month.rename('month'),testing456['business_id']])
for key, record in testgb:
    if len(testgb.get_group(key)) > 9:
        testingdf123 = testgb.get_group(key)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
